In [21]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import numpy
import copy
from qiskit.quantum_info import Pauli
from qiskit.chemistry import FermionicOperator
from qiskit.aqua.operators.legacy import op_converter
from openfermion.circuits import slater_determinant_preparation_circuit
from qiskit import QuantumCircuit,execute
from qiskit import Aer
from joblib import Parallel,delayed
from qiskit.aqua.operators import WeightedPauliOperator,Z2Symmetries
from qiskit.chemistry.components.variational_forms import UCCSD
import scipy
import time
import numba
from numba import jit

In [22]:
def countYgates(pauli_label):
    countYgates = sum(map(lambda x : 1 if 'Y' in x else 0, pauli_label))
    return countYgates
def qubitOp(h1,h2):
    qubit_op=FermionicOperator(h1,h2).mapping('parity')
    return qubit_op
def egBandHamiltonian(U):
    #Getting chemical Potential for Half-Filling
    with open('../chem_pot_for_Half_Fill.txt','r') as f:
        lines=f.readlines()[1:]
        for line in lines:
            elems=line.split()
            if int(elems[0])==U:
                muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
    #Getting the one body and two body interaction vertexes
    with open('../'+str(U)+'/v1e.dat','r') as f:
            lines=f.readlines()[1:]
            num_sites=4
            chem_pot=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1=numpy.zeros((2*num_sites,2*num_sites))
            for line in lines:
                elems=line.split()
                eg_h1[int(elems[0])][int(elems[1])]=float(elems[2])
                eg_h1[int(elems[0])+num_sites][int(elems[1])+num_sites]=float(elems[2])
            for i in range(2*num_sites):
                chem_pot[i][i]=-muHalf
            eg_h1=eg_h1+chem_pot       
    with open('../'+str(U)+'/v2e.dat','r') as f:
        num_sites=4
        eg_h2=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        for line in f:
            if "#" in line:
                continue
            line = line.split()
            i,j,k,l = map(int, line[:4])
            val = float(line[4])
            eg_h2[i,j,k,l] = eg_h2[i+num_sites,j+num_sites,k,l] = eg_h2[i,j,k+num_sites,l+num_sites]             = eg_h2[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
    qubitH=qubitOp(eg_h1,eg_h2)
    qubitH=Z2Symmetries.two_qubit_reduction(qubitH, 4)
    Hmat=op_converter.to_matrix_operator(qubitH).dense_matrix
    w,v=numpy.linalg.eigh(Hmat)
    Eg=w[0]
    state_g=v[:,0]
    return qubitH,Hmat,Eg,state_g

In [55]:
def commutator(A,B):
        B2=WeightedPauliOperator([[-B.paulis[0][0],B.paulis[0][1]]])
        return A.multiply(B).add(B2.multiply(A))
def commutatorsForGradient(Op):
    Op_conj=WeightedPauliOperator([[-1j,Op.paulis[0][1]]])
    M=Op.multiply(qubitH).add(qubitH.multiply(Op_conj))
    Mmat=op_converter.to_matrix_operator(M).matrix
    return Mmat
def commutatorPool(qubitH,stripZs=False):
    def stripZ(label):
        label=numpy.array(list(label))
        label[label=='Z']='I'
        label=''.join(list(label))
        return label
    #construct commutator pool from the Hamiltonian
    pool_H=[WeightedPauliOperator([[1j,qubitH.paulis[i][1]]]) for i in range(len(qubitH.paulis))][1:]
    #commutator between operators
    commutator_pool=[WeightedPauliOperator([[1j,commutator(op1,op2).paulis[0][1]]]) for op1 in pool_H for op2 in pool_H if countYgates(commutator(op1,op2).paulis[0][1].to_label())%2==1]
    labels=[commutator_pool[i].paulis[0][1].to_label() for i in range(len(commutator_pool))]
    unique_labels=numpy.unique(labels)
    if stripZs==True:
        new_labels=[]
        for i in range(len(unique_labels)):
            new_labels.append(stripZ(unique_labels[i]))
        unique_labels=numpy.unique(new_labels)    
    commutator_pool=[WeightedPauliOperator([[1j,Pauli(unique_labels[i])]]) for i in range(len(unique_labels))]
    print(len(commutator_pool))
    return commutator_pool
def compute_gradient(M,state):
    grad=state@M@numpy.conjugate(state)
    return grad.real
def SMO(cost,params,runs=20,tol=1e-4,save_opt_steps=False):
    index=1
    conv_err=1000
    def E_landscape(ind,ang,cost,params):
        params1=copy.deepcopy(params)
        params1[ind]=params1[ind]+ang #ang
        #circ=var_form_base.construct_circuit(parameters=params1)
        E=cost(params1)
        return E.real
    def determine_unknowns(E,cost,params,ind):
        L1=E#_landscape(ind,0,params,Hmat)
        L2=E_landscape(ind,numpy.pi/4.,cost,params)
        L3=E_landscape(ind,-numpy.pi/4.,cost,params)
        ratio=(L3-L2)/(2*L1-L2-L3)
        a3=(L2+L3)/2.
        a2=2*params[ind]-numpy.arctan(ratio)
        a1=(L1-a3)/numpy.cos(numpy.arctan(ratio))
        return a1,a2,a3
    def update(E,cost,params,ind):
        a1,a2,a3=determine_unknowns(E,cost,params,ind)
        thetaStar=a2/2.+numpy.pi/2. if a1>0 else a2/2.
        newParams=copy.deepcopy(params)
        newParams[ind]=thetaStar
        updEnergy=a3-a1 if a1>0 else a3+a1
        return newParams,updEnergy.real
    while conv_err>tol and index<runs:
        print("looped "+str(index)+" times")
        Eold=cost(params)
        init=0
        for i in range(len(params)):#[::-1][0:1]:
            #first run sequential minimial optimization (SMO)  for a given multiqubit operator using 
            #exact analytical form for cost function
            if init==0:
                E=Eold
            ind=i
            params,E=update(E,cost,params,ind)
            if save_opt_steps==True:
                with open('paramsForQubitAdapt_eg_model_parity.txt','a') as f:
                    Str=["{:0.16f}".format(params[i].real) for i in range(len(params))]
                    print('['+','.join(Str)+']'+'#'+"{:0.16f}".format(E.real),file=f)
            else:
                continue
            init=init+1  
        conv_err=Eold-E
        print("inner loop error",conv_err)
        index=index+1
    res={'fun':E,'x':params}    
    return res 
def Energy(params):
    circ=var_form_base.construct_circuit(parameters=params)
    state=execute(circ,backend,shots=1024).result().get_statevector() 
    E=(numpy.conjugate(state)@Hmat@state).real
    print(numpy.abs(state_g@numpy.conjugate(state))**2)
    if save_steps==True:
        with open('paramsForQubitAdapt_eg_model_parity.txt','+a') as f:
            Str=["{:0.16f}".format(params[i].real) for i in range(len(params))]
            print('['+','.join(Str)+']'+'#'+"{:0.16f}".format(E.real),file=f)
    return E

In [50]:
U=7
#Constructing Hamiltonian
qubitH,Hmat,Eg,state_g=egBandHamiltonian(U)
#commutator pools
commutator_pool=commutatorPool(qubitH,stripZs=True)
#constructing matrices for gradient computation
MatrixRepOfPoolOps=list(map(commutatorsForGradient,commutator_pool))
print("pool sizes",len(MatrixRepOfPoolOps))
backend=Aer.get_backend('statevector_simulator')
#preparing HF state
circ=QuantumCircuit(6)
circ.x(0)
circ.x(3)
var_form_base=UCCSD(8,num_particles=4, initial_state=circ,qubit_mapping='parity',two_qubit_reduction=True)
var_form_base.manage_hopping_operators()
error=1000

16
pool sizes 16


In [25]:
def fun_jac(params):
    circ=var_form_base.construct_circuit(parameters=params)
    state=execute(circ,backend,shots=1024).result().get_statevector() 
    def State(params):
        circ=var_form_base.construct_circuit(parameters=params)
        state=execute(circ,backend,shots=1024).result().get_statevector() 
        #E=(numpy.conjugate(state)@Hmat@state).real
        return state
    setofParams1=[numpy.array(params)+numpy.array([0]*i+[numpy.pi/2]+[0]*(len(params)-i-1)) for i in range(len(params))]
    States=numpy.array(Parallel(n_jobs=7,verbose=2)(delayed(State)(setofParams1[i]) for i in range(len(params))))#qubit_pool_UCCSD))))#
    jac=[]
    for i in range(len(States)):
        jac.append((numpy.conjugate(state)@Hmat@States[i]).imag)
    return jac

$|\Psi\rangle =\exp(i\theta_{1}A_{1})\exp(i\theta_{2}A_{2})\ldots \exp(i\theta_{n}A_{n})|0\rangle$

In [1]:
ExcOps=[]
params=[]
with open('data_eg_model_parity.txt','r') as f:
    lines=f.readlines()
    for line in lines:
        if ((line[0]=='Y') or (line[0]=='X') or (line[0]=='Z') or (line[0]=='I')):
            label=line[:-1]
            print(label)
            ExcOps.append(label)
            var_form_base.push_hopping_operator(WeightedPauliOperator([[1j,Pauli(label)]]))
    params=eval(lines[-1])        

YXIIII


NameError: name 'var_form_base' is not defined

In [45]:
ExcOps

['XIIII\n']

In [56]:
save_steps=False
Energy(params)

0.999999999960496


-19.184985040792363

In [9]:
save_steps=True
res=scipy.optimize.minimize(Energy,params,method='BFGS',options={'ftol':1e-7})
params,E=list(res['x']),res['fun']

QiskitError: 'Keyboard interrupt in parallel_map.'

In [ ]:
# params=[]
EnergyArr=[]
# ExcOps=[]
ti=time.time()
steps=35
for i in range(35):
    save_steps=False
    circ=var_form_base.construct_circuit(parameters=params)
    state=execute(circ,backend,shots=1024).result().get_statevector()
    grads=numpy.array(Parallel(n_jobs=7,verbose=2)(delayed(compute_gradient)(MatrixRepOfPoolOps[i],state) for i in range(len(commutator_pool))))#qubit_pool_UCCSD))))#
    indexes=numpy.argsort(abs(grads))[::-1][:3]
    print("three highest grads",grads[indexes[0]],grads[indexes[1]],grads[indexes[2]])
    PauliOp=commutator_pool[indexes[0]]
    ExcOps.append(PauliOp.paulis[0][1].to_label())
    print("chosen Op",ExcOps[-1])
    with open('paramsForQubitAdapt_eg_model_parity.txt','a') as f:
        print("label-",ExcOps[-1],file=f)
    params.append(0.0)
    var_form_base.push_hopping_operator(PauliOp)
    #construct circuit for estimating Hamiltonian
    circ=var_form_base.construct_circuit(parameters=params)
    res=SMO(Energy,params,runs=30,tol=1e-6,save_opt_steps=True)
    E,params=res['fun'],list(res['x'])
    save_steps=True
    res=scipy.optimize.minimize(Energy,params,method='BFGS',options={'ftol':1e-7})
    params,E=list(res['x']),res['fun']
    print("num_params",var_form_base.num_parameters)
    print("Energy",E)
    EnergyArr.append(E)
    print("time elapsed",time.time()-ti)    
    error=EnergyArr[-1]-Eg

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.


three highest grads -0.008892592605310431 -0.007382634264750995 0.00651677432365276
chosen Op IYIIIX


[Parallel(n_jobs=7)]: Done  12 out of  16 | elapsed:    1.4s remaining:    0.5s
[Parallel(n_jobs=7)]: Done  16 out of  16 | elapsed:    1.4s finished


looped 1 times
inner loop error 1.2765394117764117e-05
looped 2 times
inner loop error 9.866297087768316e-06
looped 3 times
inner loop error 8.528910587557448e-06
looped 4 times
inner loop error 8.129987207894374e-06
looped 5 times
inner loop error 7.879447554870467e-06
looped 6 times
inner loop error 7.674756556497186e-06
looped 7 times
inner loop error 7.4985967764007455e-06
looped 8 times
inner loop error 7.346826464527112e-06
looped 9 times
inner loop error 7.216430532253071e-06
looped 10 times
inner loop error 7.1042730702686185e-06
looped 11 times
inner loop error 7.007366953359906e-06
looped 12 times
inner loop error 6.923091454780206e-06
looped 13 times
inner loop error 6.849235607120363e-06
looped 14 times
inner loop error 6.783958660605549e-06
looped 15 times
inner loop error 6.725732788481764e-06
looped 16 times
inner loop error 6.673291814252025e-06
looped 17 times
inner loop error 6.625588611086641e-06
looped 18 times
inner loop error 6.581760189305896e-06
looped 19 times


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.


...

looped 1 times
inner loop error 1.0764722446765518e-12
num_params 50
Energy -19.18498504083966
time elapsed 18629.352301836014


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.


three highest grads 7.141661899617152e-07 -5.597696412676205e-07 -4.0834884379014474e-07
chosen Op IIXIYI


[Parallel(n_jobs=7)]: Done  12 out of  16 | elapsed:    0.7s remaining:    0.2s
[Parallel(n_jobs=7)]: Done  16 out of  16 | elapsed:    0.8s finished


looped 1 times
inner loop error 9.592326932761353e-13


In [2]:
params_arr=[]
Labels_arr=[]
Energy_arr=[]
count=0
with open('paramsForQubitAdapt_eg_model_parity.txt','r') as f:
    lines=f.readlines()
    for i in range(len(lines)):
        if lines[i][0]=='l':
            count=count+1
            label=lines[i].split('-')[1][1:-1]
            Labels_arr.append(label)
            if count>1:
                params,E=lines[i-1].split('#')
                params_arr.append(eval(params))
                Energy_arr.append(float(E))
    params_arr.append(eval(lines[-1]))
    Energy_arr.append(float(lines[-1].split('#')[1]))

In [13]:
params_arr=params_arr[0:33]
Energy_arr=Energy_arr[0:33]

In [20]:
for i in range(len(params_arr)):
    with open('data_eg_model_parity.txt','+a') as f:
            print(Labels_arr[i]+'\n'+'['+','.join([str(params_arr[i][j]) for j in range(len(params_arr[i]))])+']'+'#'+"{:0.10f}".format(Energy_arr[i]),file=f)